# Embeddings로 텍스트를 벡터로 변환하기

이 노트북에서는 **Embeddings**를 사용하여 텍스트를 벡터(숫자 배열)로 변환하는 방법을 알아봅니다.

## RAG 파이프라인에서의 위치

```
┌─────────────┐     ┌─────────────┐     ┌─────────────┐     ┌─────────────┐
│  📄 Load    │ ──▶ │  ✂️ Split   │ ──▶ │  🔢 Embed   │ ──▶ │  💾 Store   │
│  (로드)     │     │  (분할)     │     │  (임베딩)    │     │  (저장)     │
└─────────────┘     └─────────────┘     └─────────────┘     └─────────────┘
                                              ↑
                                           현재 단계
```

## Embedding이란?

텍스트의 **의미**를 숫자 벡터로 표현하는 것입니다.

```
"안녕하세요" → [0.12, -0.34, 0.56, ..., 0.78]  (1536차원)
```

**특징:**
- 의미가 비슷한 텍스트 → 벡터가 가까움
- 벡터 간 거리/유사도로 검색 가능

## 주요 Embedding 모델

| 모델 | 특징 |
|------|------|
| OpenAI `text-embedding-3-small` | 빠르고 저렴 |
| OpenAI `text-embedding-3-large` | 높은 정확도 |
| Ollama (nomic-embed-text) | 로컬, 무료 |
| HuggingFace | 다양한 오픈소스 모델 |

---

# 1. Ollama 설치 및 서버 실행

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

# 2. 임베딩 모델 다운로드 & 패키지 설치

In [ ]:
# nomic-embed-text: Ollama의 임베딩 모델
!ollama pull nomic-embed-text
!pip install -q langchain-ollama

# 3. Embeddings 생성

**코드 설명:**

```python
model = OllamaEmbeddings(model='nomic-embed-text')
embeddings = model.embed_documents(['텍스트1', '텍스트2', ...])
```

- `embed_documents()` - 여러 문서를 벡터로 변환
- `embed_query()` - 검색 쿼리를 벡터로 변환

In [ ]:
from langchain_ollama import OllamaEmbeddings

# 임베딩 모델 생성
model = OllamaEmbeddings(model='nomic-embed-text')

# 여러 문서 임베딩
texts = [
    'Hi there!',
    'Oh, hello!',
    "What's your name?",
    'My friends call me World',
    'Hello World!'
]

embeddings = model.embed_documents(texts)

print(f"임베딩된 문서 수: {len(embeddings)}")
print(f"각 벡터의 차원: {len(embeddings[0])}")

# 4. 임베딩 벡터 확인

In [ ]:
# 첫 번째 텍스트의 벡터 (일부만 출력)
print(f"텍스트: '{texts[0]}'")
print(f"벡터 (처음 10개): {embeddings[0][:10]}")
print(f"...")
print(f"벡터 (마지막 5개): {embeddings[0][-5:]}")

# 5. 유사도 계산

In [ ]:
import numpy as np

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# 각 텍스트 쌍의 유사도 계산
print("=== 텍스트 간 유사도 ===")
for i in range(len(texts)):
    for j in range(i+1, len(texts)):
        sim = cosine_similarity(embeddings[i], embeddings[j])
        print(f"'{texts[i]}' vs '{texts[j]}': {sim:.4f}")

# 6. 쿼리 임베딩 (검색용)

In [ ]:
# 검색 쿼리 임베딩
query = "greeting"
query_embedding = model.embed_query(query)

print(f"쿼리: '{query}'")
print(f"쿼리 벡터 차원: {len(query_embedding)}")

# 쿼리와 각 문서의 유사도
print("\n=== 쿼리와 문서 유사도 ===")
for i, text in enumerate(texts):
    sim = cosine_similarity(query_embedding, embeddings[i])
    print(f"'{text}': {sim:.4f}")

---

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본 (OpenAI)
from langchain_openai import OpenAIEmbeddings
model = OpenAIEmbeddings(model='text-embedding-3-small')

# 변경 (Ollama)
from langchain_ollama import OllamaEmbeddings
model = OllamaEmbeddings(model='nomic-embed-text')
```

## Embedding 메서드

| 메서드 | 용도 |
|--------|------|
| `embed_documents([texts])` | 문서들을 벡터로 변환 (저장용) |
| `embed_query(text)` | 검색 쿼리를 벡터로 변환 |

## 다음 단계

임베딩된 벡터는 **Vector Store**에 저장하여 유사도 검색에 사용합니다. (09번 노트북)